In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
train_dataset = np.genfromtxt('resources/train.csv', dtype=np.float32, delimiter=',')
train_targets = train_dataset[:,-1]
train_targets = np.array([1 if x == 2 else 0 for x in train_targets])
test_dataset = np.genfromtxt('resources/test.csv', dtype=np.float32, delimiter=',')
test_targets = test_dataset[:,-1]
test_targets = np.array([1 if x == 2 else 0 for x in test_targets])
combined_dataset=np.concatenate((train_dataset[:,:-1], test_dataset[:,:-1]))

In [3]:
np.bincount(train_targets)

array([58630, 67343])

In [4]:
numeric_indices = []
for i in range(combined_dataset.shape[1]):
    n_unique = len(np.unique(combined_dataset[:, i]))
    numeric_indices.append(n_unique > 2)
print (len(numeric_indices))
numeric_indices = np.array(numeric_indices, dtype=np.bool)

122


In [5]:
combined_dataset[:, numeric_indices] = StandardScaler().fit_transform(
                                        combined_dataset[:, numeric_indices]
                                        )

In [21]:
batch_size = 1500
num_feature = 122
num_labels = 2
graph = tf.Graph()
with graph.as_default():
    target = tf.placeholder(tf.float32, shape=(None, 2))
    n_input = num_feature
    n_hidden_1 = 200
    n_hidden_2 = 100
    
    inputs = {'l1': tf.placeholder(tf.float32, shape=(None, num_feature)),
              'l2': tf.placeholder(tf.float32, shape=(None, n_hidden_1))}
    
    targets = {'l1': tf.placeholder(tf.float32, shape=(None, num_feature)),
               'l2': tf.placeholder(tf.float32, shape=(None, n_hidden_1))}

    weights = {'l1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], dtype=tf.float32)),
               'l2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], dtype=tf.float32))}

    enc_biases = {'l1': tf.Variable(tf.truncated_normal([n_hidden_1], dtype=tf.float32)),
                  'l2': tf.Variable(tf.truncated_normal([n_hidden_2], dtype=tf.float32))}
    
    dec_biases = {'l1': tf.Variable(tf.truncated_normal([n_input], dtype=tf.float32)),
                  'l2': tf.Variable(tf.truncated_normal([n_hidden_1], dtype=tf.float32))}
    
    def encode(layer_id):
        return tf.add(tf.matmul(inputs[layer_id], weights[layer_id]), enc_biases[layer_id])
    
    def decode(layer, layer_id):
        return tf.add(tf.matmul(layer, tf.transpose(weights[layer_id])), dec_biases[layer_id])
    
    def get_loss(inp, out):
        difference = tf.sub(inp, out) 
        return tf.sqrt(tf.reduce_mean(tf.square(difference)))   

    def encoder_loss(layer_id):
        enc = encode(layer_id)
        dec = decode(enc, layer_id)
        loss = get_loss(targets[layer_id], dec)
        return loss
    
    loss_l1 = encoder_loss('l1')
    optimizer_l1 = tf.train.AdamOptimizer(0.001).minimize(loss_l1)
    loss_l2 = encoder_loss('l2')
    optimizer_l2 = tf.train.AdamOptimizer(0.001).minimize(loss_l2)
#     def training(loss):
#         optimizer = tf.train.AdamOptimizer(0.001)
#         global_step = tf.Variable(0, name='global_step', trainable=False)
#         return optimizer.minimize(loss, global_step=global_step)
#     optimizer = tf.train.AdamOptimizer(0.001).minimize(train())
#     layer_1_f = tf.nn.tanh(tf.add(tf.matmul(clean_data, wh1), b1))
#     wy = tf.Variable(tf.truncated_normal([n_hidden_1, num_labels], dtype=tf.float32))
#     by = tf.Variable(tf.truncated_normal([num_labels], dtype=tf.float32))
#     y_ = tf.matmul(layer_1_f, wy) + by
#     a_ = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(target,1), tf.argmax(y_,1)), tf.float32))    
#     loss_ = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, target))
#     optimize_ = tf.train.AdamOptimizer(0.0001).minimize(loss_)    

In [24]:
num_steps = 7000
randomized_dataset = combined_dataset.copy()
np.random.shuffle(randomized_dataset)
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    for step in range(1, num_steps + 1):
        offset = (step * batch_size) % (randomized_dataset.shape[0] - batch_size)
        batch_data = randomized_dataset[offset:(offset + batch_size), :]
        noisy_batch_data = batch_data.copy()
        for i, point in enumerate(noisy_batch_data):
            noisy_batch_data[i, :] = point + np.random.normal(0, 1, noisy_batch_data.shape[1])
        feed_dict = {inputs['l1']: noisy_batch_data, targets['l1']: batch_data}
        _, l = session.run([optimizer_l1, loss_l1], feed_dict=feed_dict)
        if step % 500 == 0:
            print step, ':', l

Initialized
500 : 89.5356
1000 : 37.55
1500 : 16.0799
2000 : 6.98832
2500 : 3.52445
3000 : 1.2815
3500 : 0.585303
4000 : 0.391985
4500 : 0.32777
5000 : 0.310576
5500 : 0.596602
6000 : 0.3331
6500 : 0.294123
7000 : 0.300371


In [26]:
num_steps = 2000
preds = []
randomized_dataset = combined_dataset.copy()
np.random.shuffle(randomized_dataset)
new_input=[]
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    encoder_op = encode('l1')
    encoded_randomized_dataset = encoder_op.eval(feed_dict={inputs['l1']: randomized_dataset})
    for step in range(1, num_steps + 1):
        offset = (step * batch_size) % (randomized_dataset.shape[0] - batch_size)
        batch_data = encoded_randomized_dataset[offset:(offset + batch_size), :]
        noisy_batch_data = randomized_dataset[offset:(offset + batch_size), :].copy()
        for i, point in enumerate(noisy_batch_data):
            noisy_batch_data[i, :] = point + np.random.normal(0, 1, noisy_batch_data.shape[1])
        noisy_batch_data = encoder_op.eval(feed_dict={inputs['l1']: noisy_batch_data})
        feed_dict = {inputs['l2']: noisy_batch_data, targets['l2']: batch_data}
        _, l = session.run([optimizer_l2, loss_l2], feed_dict=feed_dict)
        if step % 500 == 0:
            print step, ':', l

Initialized
500 : 506.473
1000 : 199.229
1500 : 69.1318
2000 : 19.6181


In [79]:
num_steps = 7000
preds = []
randomized_dataset = combined_dataset.copy()
#     np.random.shuffle(randomized_dataset)
new_input=[]
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (randomized_dataset.shape[0] - batch_size)
        batch_data = randomized_dataset[offset:(offset + batch_size), :]
        noisy_batch_data = batch_data.copy()
        for i, point in enumerate(noisy_batch_data):
            noisy_batch_data[i, :] = point + np.random.normal(0, 1, noisy_batch_data.shape[1])
        feed_dict = {input_data: noisy_batch_data, clean_data: batch_data}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step", step, ":", l)
#             print "SKLEARN loss", np.sqrt(mean_squared_error(predictions, batch_data))
    new_input=layer_1.eval(feed_dict = {input_data :combined_dataset})
    test_targets_1hot = np.array([(x == np.arange(2)).astype(np.float32) for x in test_targets])
#     print(test_targets_1hot[:5])
    for step in range(370):
        c = np.random.choice(train_targets.shape[0], 2000, replace=False)
        batch_data = train_dataset[:,:-1][c]
        batch_targets = train_targets[c]
        batch_targets=np.array([(x == np.arange(2)).astype(np.float32) for x in batch_targets])
        feed_dict = {clean_data:batch_data, target:batch_targets}
        l, _ = session.run([loss_, optimize_], feed_dict=feed_dict)
        if (step % 10 == 0):
            print("Minibatch logloss at step", step, ":", l)
    new_input = layer_1_f.eval(feed_dict={clean_data:combined_dataset})
    print(session.run(a_, feed_dict={clean_data:combined_dataset[len(train_targets):],target:test_targets_1hot}))

Initialized
('Minibatch loss at step', 0, ':', 222.77254)
('Minibatch loss at step', 500, ':', 89.292938)
('Minibatch loss at step', 1000, ':', 39.540432)
('Minibatch loss at step', 1500, ':', 16.709871)
('Minibatch loss at step', 2000, ':', 26.412645)
('Minibatch loss at step', 2500, ':', 3.530025)
('Minibatch loss at step', 3000, ':', 1.5838422)
('Minibatch loss at step', 3500, ':', 0.6859898)
('Minibatch loss at step', 4000, ':', 0.40181309)
('Minibatch loss at step', 4500, ':', 0.38369343)
('Minibatch loss at step', 5000, ':', 0.31740069)
('Minibatch loss at step', 5500, ':', 0.33525366)
('Minibatch loss at step', 6000, ':', 0.36136743)
('Minibatch loss at step', 6500, ':', 0.29976884)
('Minibatch logloss at step', 0, ':', 2.0832458)
('Minibatch logloss at step', 10, ':', 0.43901727)
('Minibatch logloss at step', 20, ':', 0.27650648)
('Minibatch logloss at step', 30, ':', 0.19391036)
('Minibatch logloss at step', 40, ':', 0.22965193)
('Minibatch logloss at step', 50, ':', 0.1282324

In [36]:
#saving new train & test data 
from collections import Counter
print(train_dataset.shape)
new_train_data=new_input[:train_dataset.shape[0]]
new_train_data=np.c_[new_train_data,train_targets]
new_test_data=new_input[train_dataset.shape[0]:]
new_test_data=np.c_[new_test_data, test_targets]
print(new_train_data.shape, new_test_data.shape)
# np.savez('out/new_input.npz',train=new_train_data, test=new_test_data)

(125973, 123)
((125973, 201), (22544, 201))


In [37]:
model = LogisticRegression(C=1,class_weight='balanced',n_jobs=-1)

In [37]:
print (cross_val_score(model, new_train_data[:, :-1], train_targets, cv=3, scoring='precision_macro').mean())

0.731867870076


In [45]:
new_train_data = np.load('out/new_input.npz')['train']
new_test_data = np.load('out/new_input.npz')['test']

In [38]:
model.fit(new_train_data[:, :-1], train_targets)

LogisticRegression(C=1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [39]:
preds = model.predict(new_test_data[:, :-1])
# np.savez('out/preds.npz',preds=preds)

In [40]:
print(accuracy_score(preds, test_targets))

0.740684882896


In [41]:
print(confusion_matrix(test_targets, preds))

[[7736 5097]
 [ 749 8962]]
